In [381]:
import os
os.getcwd()

'/Users/jy/Desktop/corpus'

In [1]:
ls

Untitled.ipynb               corpus_secte.txt
concat_corpus_secte/         corpus_taosime.txt
concat_sous_corpus_religion/ corpus_yoga.txt
contre_corpus/               nettoyage_corpus_py/
corpus/                      secte_activites.txt
corpus_bouddhisme.txt        secte_livres.txt
corpus_religion.txt          yoga.txt


In [2]:
#tsv file (3 columns; text, partition, label)
#txt file (each line represents a word of the vocabulary)

In [32]:
import spacy
nlp = spacy.load("fr_core_news_md")

In [13]:
from spacy.lang.fr.stop_words import STOP_WORDS

In [382]:
from pandas.core.frame import DataFrame
import pandas as pd

In [383]:
import re

In [384]:
def delete_dot_ds_store(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            if filename == '.DS_Store':
                os.remove(os.path.join(root, filename))

In [418]:
def sample_text (path): 
    sample_list = []#sample_list = est pour conserver tous les textes avec le format de liste
    filenames = os.listdir(path)
    for filename in filenames:
        filepath = path+os.sep+filename
        f = open(filepath,'r',encoding='utf-8')
        text = f.read()
        doc = nlp(text)

        token_list = []
        for token in doc:
            token_list.append(str.lower(token.lemma_))#tokenizer la phrase et la conserver dans token_list
        filtered_sentence =[] 

        for word in token_list:
            lexeme = nlp.vocab[word]
            if lexeme.is_stop == False:
                if word.isalpha() == True and len(word)>1:
                    #print(word)
                    word=re.sub('[\u4e00-\u9fa5]', '', word)#supprimer les caractères chinois
                    filtered_sentence.append(word)#supprimer les mots vides et la conserver dans filtered_sentence(list)
        one_sample = ''
        for token in filtered_sentence:
            if token != '':
                one_sample += token+' '
        sample_list.append(one_sample)
    return sample_list

def label(sujet,sample_list):
    label = sujet
    label_list = []
    for i in range(len(sample_list)):
        label_list.append(label)
    return label_list

def partition(sample_list):
    partition = []
    nb_train = int(len(sample_list)*0.6)
    nb_test = int(len(sample_list)*0.2)
    nb_valid = len(sample_list)-nb_train-nb_test
    for i in range(nb_train):
        partition.append('train')
    for i in range(nb_test):
        partition.append('test')
    for i in range(nb_valid):
        partition.append('val')
    return partition

In [402]:
path = '/Users/jy/Desktop/corpus/corpus/activites'
list_activites_sectaire = sample_text (path) 
label_activites_sectaire = label('secte',list_activites_sectaire)
partition_activites_sectaire = partition(list_activites_sectaire)

In [403]:
path = '/Users/jy/Desktop/corpus/corpus/livres'
list_livres_sectaire = sample_text (path) 
label_livres_sectaire = label('secte',list_livres_sectaire)
partition_livres_sectaire = partition(list_livres_sectaire)

In [404]:
path = '/Users/jy/Desktop/corpus/contre_corpus/bouddhisme'
list_bouddhisme = sample_text (path) 
label_bouddhisme = label('religion',list_bouddhisme)
partition_bouddhisme = partition(list_bouddhisme)

In [405]:
path = '/Users/jy/Desktop/corpus/contre_corpus/yoga'
list_yoga = sample_text (path) 
label_yoga = label('religion',list_yoga)
partition_yoga = partition(list_yoga)

In [406]:
path = '/Users/jy/Desktop/corpus/contre_corpus/taoisme'
list_taoisme = sample_text (path) 
label_taoisme = label('religion',list_taoisme)
partition_taoisme = partition(list_taoisme)

In [407]:
list_text = list_activites_sectaire + list_livres_sectaire + list_bouddhisme + list_yoga + list_taoisme

In [408]:
list_label = label_activites_sectaire + label_livres_sectaire + label_bouddhisme + label_yoga + label_taoisme

In [409]:
list_partition = partition_activites_sectaire + partition_livres_sectaire + partition_bouddhisme + partition_yoga + partition_taoisme

In [410]:
len(list_text) == len(list_label) == len(list_partition)

True

In [411]:
data_all={"text" : list_text,
           "partition" : list_partition,
           "label" : list_label}
data=DataFrame(data_all)
print(data)


                                                   text partition     label
0     japon jeune pratiquant école minghui célébrer ...     train     secte
1     état unis sénateur représentant chambre envoye...     train     secte
2     indonésie pratiquant falun dafa conférence par...     train     secte
3     washington dc célébration journée mondial falu...     train     secte
4     new york membre congrès émettre certificat pro...     train     secte
...                                                 ...       ...       ...
1271  livre voie vertu siècle niais activité superfl...       val  religion
1272  empire nommer tao thsiouen fréquenter montagne...       val  religion
1273  livre voie vertu chapitre lv posséder vertu so...       val  religion
1274  livre voie vertu regarde raison profond abstra...       val  religion
1275  grand guerrier temps ancien porte coup préfére...       val  religion

[1276 rows x 3 columns]


In [412]:
data[:-10]

text partition     label
0     japon jeune pratiquant école minghui célébrer ...     train     secte
1     état unis sénateur représentant chambre envoye...     train     secte
2     indonésie pratiquant falun dafa conférence par...     train     secte
3     washington dc célébration journée mondial falu...     train     secte
4     new york membre congrès émettre certificat pro...     train     secte
...                                                 ...       ...       ...
1261  chapitre arme excellent instrument malheur hom...       val  religion
1262  tsen chang kong antérieur ère chrétien or tao ...       val  religion
1263  chapitre homme vertu supérieur eau eau excell ...       val  religion
1264  livre voie vertu notice historique lao tseu xi...       val  religion
1265  chose monde poindre mou faible eau briser dur ...       val  religion

[1266 rows x 3 columns]

In [413]:
data.to_csv("./corpus.tsv", index=False, sep='\t', encoding='utf-8')

In [414]:
list_vocabulary = []
for i in range(len(list_text)):
    for word in list_text[i].split(' '):
        if word not in list_vocabulary and word != '':
            list_vocabulary.append(word)

In [415]:
f = open('/Users/jy/Desktop/corpus/vocabulary.txt','w',encoding = "utf-8")
for word in list_vocabulary:
    f.write(word+'\n')
f.close()

In [416]:
len(list_vocabulary)

25011